In [ ]:
"""
Evaluate IMO_try.csv with DeepSeek‑R1‑671B via HKUST streaming API
and save answer.csv
Prerequisites
  pip install requests pandas
"""
import re, time, json, requests, pandas as pd

# ── 1. API setup ──────────────────────────────────────────────
KEY        = "74fec51cd1b74a919057a11d48f0cc523d866d9aae3846f983f4387de1f91b0b"                 # <‑‑ put your key here
API_URL    = "https://gpt-api.hkust-gz.edu.cn/v1/chat/completions"
HEADERS    = {
    "Content-Type": "application/json",
    "Authorization": f"Bearer {KEY}",
}
MODEL_NAME  = "DeepSeek-R1-671B"
PROMPT_TAIL = ("\n\nPlease think step by step, and put your final answer within "
               "\\boxed{}.\n\n")
PAUSE_S     = 1.0

boxed_re = re.compile(r"\\boxed\{([^}]*)\}")

def extract_boxed(text: str) -> str | None:
    m = boxed_re.findall(text)
    return m[-1].strip() if m else None

def ask_deepseek(question: str) -> str:
    payload = {
        "model": MODEL_NAME,
        "messages": [
            {"role": "system",
             "content": "You are an expert mathematician. Please solve the problem."},
            {"role": "user",
             "content": question + PROMPT_TAIL},
        ],
        "stream": True
    }
    response = requests.post(API_URL, headers=HEADERS,
                             data=json.dumps(payload), stream=True)

    parts = []
    for raw in response.iter_lines():
        if not raw:
            continue
        chunk = raw.decode("utf-8").lstrip("data: ").strip()
        if chunk == "[DONE]":
            break
        try:
            obj = json.loads(chunk)
        except json.JSONDecodeError:
            continue
        delta = obj["choices"][0]["delta"]
        txt   = delta.get("content")
        if txt:
            parts.append(txt)
        if obj["choices"][0].get("finish_reason") == "stop":
            break
    return "".join(parts)

# ── 2. load dataset ───────────────────────────────────────────
CSV_PATH = "IMO_try_5.csv"
df       = pd.read_csv(CSV_PATH)

df = df.rename(columns={
    "problem_id": "problem_id",
    "problem":    "problem",
    "answer":     "correct_answer",
})[["problem_id", "problem", "correct_answer"]]



In [ ]:

# ── 3. inference loop ────────────────────────────────────────
rows = []
COUNT = 0
for row in df.itertuples(index=False):
    COUNT += 1
    print("Solving Problem ", COUNT, "...")
    pid, statement, gold = row
    try:
        reply = ask_deepseek(statement)
        time.sleep(PAUSE_S)
        pred  = extract_boxed(reply)
    except Exception as err:
        pred  = f"ERROR: {err}"
    rows.append((pid, gold, pred))
    print("Problem ", COUNT, "done.")

# ── 4. save answers ──────────────────────────────────────────
out = pd.DataFrame(rows,
                   columns=["problem_id", "correct_answer", "llm_answer"])
out.to_csv("answer.csv", index=False)
print("✅  Saved", len(out), "rows to answer.csv")

In [2]:
import pandas as pd

# Load the CSV
df = pd.read_csv("imo_proof.csv")

# Write out as JSONL (one record per line)
df.to_json("ReIMO-Proof.jsonl", orient="records", lines=True)
